In [1]:
"""Correcting cash balances with manual journal entries

Demonstration of how to model manual journal entries in LUSID

Attributes
----------
reconciliations
cocoon
holdings
transaction configuration
cancel transactions
"""

'Correcting cash balances with manual journal entries\n\nDemonstration of how to model manual journal entries in LUSID\n\nAttributes\n----------\nreconcilliations\ncocoon\nholdings\n'

## Correcting cash balances with manual journal entries

In this notebook, we demonstrate how users can create manual journal entries in LUSID. For the purposes of this notebook, we will consider the scenario where a portfolio's custodian has included a stock exchange fee of £5000 in its GBP cash balance calculation. The same fee has not been included in the IBOR. This might lead a portfolio manager to go into overdraft if they trade on that amount. Therefore we create a manual entry in LUSID while the reconcilations team investigate the break.

### Setup LUSID

In [1]:
# Import LUSID
import lusid.models as models
from lusidjam import RefreshingToken
import lusid
import lusidtools.cocoon.cocoon as cocoon
from lusidtools.cocoon.utilities import create_scope_id
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.cocoon_printer import format_transactions_response
from lusidtools.cocoon.transaction_type_upload import (
    create_transaction_type_configuration,
)

# Import Libraries
import pprint
import pytz
import pandas as pd
import numpy as np
import json
import requests
import os
import warnings
from datetime import datetime, timedelta, time

# Configure notebook logging and warnings
import logging

logger = logging.getLogger()
logger.setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

print("LUSID Environment Initialised")
print(
    "LUSID API Version: ",
    api_factory.build(lusid.api.ApplicationMetadataApi)
    .get_lusid_versions()
    .build_version,
)

LUSID Environment Initialised
LUSID API Version:  0.5.4408.0


### 1) Prepare setup data

In this notebook we have a portfolio called GLOBAL-EQUITY. 

In [2]:
# Portfolio code
portfolio_code = "GLOBAL-EQUITY"

# Load a mapping file for loading data
with open(r"config/seed_data.json") as mappings_file:
    seed_data_mapping = json.load(mappings_file)

# Load a file to format holding response
with open(r"config/format_holdings_response.json") as mappings_file:
    format_holdings_response = json.load(mappings_file)

# Load transaction file
transactions_file = r"data/manual_cash_data.csv"
transactions_df = pd.read_csv(transactions_file)

# Define transaction portfolio API
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)

In [3]:
# Define function to load holdings DF


def get_holdings_df(scope, code, date=datetime.now(pytz.UTC)):

    holdings_response = transaction_portfolios_api.get_holdings(
        scope=scope, code=code, property_keys=["Instrument/default/Name"]
    )

    holdings_df = lusid_response_to_data_frame(
        holdings_response, rename_properties=True
    )

    return holdings_df

### 2) Load IBOR data

We have transactions from the IBOR which we load into the <b>ibor-nb</b> `scope`.

In [4]:
# The seed_data() function takes a file of transaction data
# and loads portfolios, instruments, and transactions into LUSID
# We use this function as a quick way of generating a demo portfolio

ibor_df = transactions_df[transactions_df["scope"] == "IBOR"]
ibor_df.drop(columns=["scope"], inplace=True)

ibor_scope = "ibor-nb123"

seed_data_response = seed_data(
    api_factory,
    ["portfolios", "instruments", "transactions"],
    ibor_scope,
    ibor_df,
    "DataFrame",
    mappings=seed_data_mapping,
)

print(
    f"Portfolio {portfolio_code} has been created in scope {ibor_scope} with transactions."
)

Portfolio GLOBAL-EQUITY has been created in scope ibor-nb123 with transactions.


Show IBOR holdings:

* The IBOR has 500,000 GBP in Cash

In [5]:
get_holdings_df(ibor_scope, "GLOBAL-EQUITY")

,instrument_uid,sub_holding_keys,Name(default-Properties),SourcePortfolioId(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency
0,LUID_ATFGUBHS,{},Aviva,ibor-nb123/GLOBAL-EQUITY,P,100000.0,100000.0,100000.0,GBP,0.0,GBP
1,LUID_STGB38I6,{},Barclays,ibor-nb123/GLOBAL-EQUITY,P,100000.0,100000.0,100000.0,GBP,0.0,GBP
2,LUID_PVOJGULG,{},BP,ibor-nb123/GLOBAL-EQUITY,P,300000.0,300000.0,300000.0,GBP,0.0,GBP
3,CCY_GBP,{},CCY_GBP,ibor-nb123/GLOBAL-EQUITY,B,500000.0,500000.0,500000.0,GBP,0.0,GBP


### 3) Load Custodian data

We have transactions from the IBOR which we load into the <b>custodian-nb</b> `scope`.

In [6]:
# The seed_data() function takes a file of transaction data
# and loads portfolios, instruments, and transactions into LUSID
# We use this function as a quick way of generating a demo portfolio

custodian_df = transactions_df[transactions_df["scope"] == "Custodian"]
custodian_df.drop(columns=["scope"], inplace=True)

custodian_scope = "custodian-nb123"

seed_data_response = seed_data(
    api_factory,
    ["portfolios", "instruments", "transactions"],
    custodian_scope,
    custodian_df,
    "DataFrame",
    mappings=seed_data_mapping,
)

print(
    f"Portfolio {portfolio_code} has been created in scope {custodian_scope} with transactions."
)

Portfolio GLOBAL-EQUITY has been created in scope custodian-nb123 with transactions.


Show Custodian holdings:

* The Custodian has 595,000 GBP in Cash

In [7]:
get_holdings_df(custodian_scope, "GLOBAL-EQUITY")

,instrument_uid,sub_holding_keys,Name(default-Properties),SourcePortfolioId(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency
0,LUID_ATFGUBHS,{},Aviva,custodian-nb123/GLOBAL-EQUITY,P,100000.0,100000.0,100000.0,GBP,0.0,GBP
1,LUID_STGB38I6,{},Barclays,custodian-nb123/GLOBAL-EQUITY,P,100000.0,100000.0,100000.0,GBP,0.0,GBP
2,LUID_PVOJGULG,{},BP,custodian-nb123/GLOBAL-EQUITY,P,300000.0,300000.0,300000.0,GBP,0.0,GBP
3,CCY_GBP,{},CCY_GBP,custodian-nb123/GLOBAL-EQUITY,B,495000.0,495000.0,495000.0,GBP,0.0,GBP


### 4) Reconcile Custodian versus IBOR

We use LUSID's holdings [reconcilation functionality](https://support.finbourne.com/how-do-i-reconcile-my-holdings-in-lusid) to reconcile the IBOR's view against the Custodian's view of GLOBAL-EQUITY. 

In [8]:
def run_ibor_cust_recon(statement_datetime):

    ibor_portfolio = models.PortfolioReconciliationRequest(
        portfolio_id=models.ResourceId(scope=ibor_scope, code=portfolio_code),
        effective_at=statement_datetime,
        as_at=statement_datetime,
    )

    # Define our fund accountant portfolio
    custodian_portfolio = models.PortfolioReconciliationRequest(
        portfolio_id=models.ResourceId(scope=custodian_scope, code=portfolio_code),
        effective_at=statement_datetime,
        as_at=statement_datetime,
    )

    # Create our reconciliation request
    reconcile_holdings_request = models.PortfoliosReconciliationRequest(
        left=ibor_portfolio,
        right=custodian_portfolio,
        instrument_property_keys=["Instrument/default/Name"],
    )

    # Reconcile the two portfolios
    reconciliation = api_factory.build(lusid.api.ReconciliationsApi).reconcile_holdings(
        portfolios_reconciliation_request=reconcile_holdings_request
    )

    return reconciliation

In [9]:
first_recon_datetime = datetime.now(pytz.UTC)
recon_result = run_ibor_cust_recon(first_recon_datetime)

print(f"The AsAt time for the recon is: {first_recon_datetime}")

The AsAt time for the recon is: 2020-05-14 09:19:35.124358+00:00


### 5) Result: we have a break of £5000!

In [10]:
lusid_response_to_data_frame(recon_result, rename_properties=True)

,instrument_uid,sub_holding_keys,left_units,right_units,difference_units,left_cost.amount,left_cost.currency,right_cost.amount,right_cost.currency,difference_cost.amount,difference_cost.currency,instrument_properties.0.key,instrument_properties.0.value.label_value,instrument_properties.0.effective_from
0,CCY_GBP,{},500000.0,495000.0,-5000.0,500000.0,GBP,495000.0,GBP,-5000.0,GBP,Instrument/default/Name,CCY_GBP,0001-01-01 00:00:00+00:00


### 6) Create a manual journal entry to correct the break

In [11]:
transactions_file = r"data/break_correction.csv"
break_correction_df = pd.read_csv(transactions_file)
break_correction_df["portfolio_code"] = portfolio_code
break_correction_df["txn_type"] = "ManualEntryCashOut"
break_correction_df.drop(columns=["scope"], inplace=True)
break_correction_df

,portfolio_code,portfolio_name,portfolio_base_currency,instrument_type,instrument_id,name,txn_id,txn_type,txn_trade_date,txn_settle_date,txn_units,txn_price,txn_consideration,currency,cash_transactions
0,GLOBAL-EQUITY,A generic global Equity portfolio,GBP,cash,GBP,Manual adjustment,cash_003,ManualEntryCashOut,01/05/2020,03/05/2020,5000,1,5000,GBP,GBP


In [12]:
transaction_mapping = {
    "identifier_mapping": {
        "ClientInternal": "instrument_id",
        "Currency": "cash_transactions",
    },
    "required": {
        "code": "portfolio_code",
        "transaction_id": "txn_id",
        "type": "txn_type",
        "transaction_price.price": "txn_price",
        "transaction_price.type": "$Price",
        "total_consideration.amount": "txn_consideration",
        "units": "txn_units",
        "transaction_date": "txn_trade_date",
        "total_consideration.currency": "currency",
        "settlement_date": "txn_settle_date",
    },
    "optional": {},
    "properties": [],
}

In [13]:
result = cocoon.load_from_data_frame(
    api_factory=api_factory,
    scope=ibor_scope,
    data_frame=break_correction_df,
    mapping_required=transaction_mapping["required"],
    mapping_optional=transaction_mapping["optional"],
    file_type="transactions",
    identifier_mapping=transaction_mapping["identifier_mapping"],
    property_columns=transaction_mapping["properties"],
    properties_scope=ibor_scope,
)

succ, failed = format_transactions_response(result)
print(f"number of successful portfolios requests: {len(succ)}")
print(f"number of failed portfolios requests    : {len(failed)}")

number of successful portfolios requests: 1
number of failed portfolios requests    : 0


### 7) Create a new transaction type for the journal entry

In [14]:
movement = [
    models.TransactionConfigurationMovementDataRequest(
        movement_types="CashAccrual",
        side="Side1",
        direction=-1,
        properties={},
        mappings=[],
    )
]


alias = models.TransactionConfigurationTypeAlias(
    type="ManualEntryCashOut",
    description="Booking of manual cash out ledgder entry",
    transaction_class="JournalEntry",
    transaction_group="default",
    transaction_roles="Shorter",
)

response = create_transaction_type_configuration(api_factory, alias, movement)

### 8) Rerun the reconcilation

The result is empty - there are no breaks.

In [15]:
second_recon_datetime = datetime.now(pytz.UTC)
second_recon_response = run_ibor_cust_recon(second_recon_datetime).values

print(second_recon_response)
print(f"The AsAt time for the recon is: {second_recon_datetime}")

[]
The AsAt time for the recon is: 2020-05-14 09:19:36.851025+00:00


### Cleanup

Cancel the adjustment to keep notebook idempotent.

In [16]:
cancel_response = transaction_portfolios_api.cancel_transactions(
    scope=ibor_scope, code=portfolio_code, transaction_ids=["cash_003"]
)

cancel_time = cancel_response.as_at

print(f"The  cancel datetime request is: {cancel_time}")

The  cancel datetime request is: 2020-05-14 09:19:37.537570+00:00
